In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import pickle
from sklearn.base import BaseEstimator
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, f1_score, precision_recall_curve, auc, ConfusionMatrixDisplay, PrecisionRecallDisplay, classification_report, roc_auc_score
from scikitplot.metrics import plot_roc

In [2]:
# Creo una clase para poder utilizar los estimadores en un gridsearch
class ClassifierWrapper(BaseEstimator):
    """
    A Custom Wrapper that can switch between classifiers.
    """ 
    def __init__(self, estimator=None):
        self.estimator = estimator

    def fit(self, X, y=None, **kwargs):
        self.estimator.fit(X, y)
        return self

    def predict(self, X, y=None):
        return self.estimator.predict(X)

    def predict_proba(self, X):
        return self.estimator.predict_proba(X)

    def score(self, X, y):
        return self.estimator.score(X, y)

In [3]:
location = "./WA_Fn-UseC_-Telco-Customer-Churn.csv"
pd.set_option("display.max_colwidth",200)
df = pd.read_csv(location,encoding = "utf-8")

In [4]:
def limpieza_df(data):
        continuas = ['tenure','monthlycharges','totalcharges']
        discretas = ['gender', 'seniorcitizen', 'partner', 'dependents', 
        'phoneservice', 'multiplelines', 'internetservice', 'onlinesecurity',
        'onlinebackup', 'deviceprotection', 'techsupport', 'streamingtv',
        'streamingmovies', 'contract', 'paperlessbilling', 'paymentmethod',
        'churn'] 
        customer_id = []
        data.columns= data.columns.str.lower()
        for i in continuas:
            data.drop(data[data[i]==' '].index,inplace=True)
            data[i] = data[i].apply(lambda x: float(x))
        customer_id = data.customerid
        data.drop(columns=['customerid'],axis=1,inplace=True)
        df_dummies = pd.get_dummies(data,drop_first=True)
        df_dummies.columns = df_dummies.columns.str.lower()
        df_dummies = pd.DataFrame(df_dummies)
        customerid =pd.DataFrame(customer_id)
        df_dummies = customerid.join(df_dummies,how='left')
        df_dummies_internet1 = df_dummies.loc[(df_dummies['internetservice_no'] == 0)]
        return df_dummies_internet1

In [5]:
data = limpieza_df(df)

In [7]:
data.drop(columns=['multiplelines_no phone service','onlinesecurity_no internet service','onlinebackup_no internet service','deviceprotection_no internet service','techsupport_no internet service','streamingtv_no internet service','streamingmovies_no internet service'],inplace=True)

In [7]:
# data_df = df.to_json()

In [10]:
# r = requests.get("http://127.0.0.1:5000/data",json=data_df)

In [11]:
# r=_

In [54]:
# r

,customerid,seniorcitizen,tenure,monthlycharges,totalcharges,gender_male,partner_yes,dependents_yes,phoneservice_yes,multiplelines_no phone service,...,streamingtv_yes,streamingmovies_no internet service,streamingmovies_yes,contract_one year,contract_two year,paperlessbilling_yes,paymentmethod_credit card (automatic),paymentmethod_electronic check,paymentmethod_mailed check,churn_yes
0,7590-VHVEG,0,1.0,29.85,29.85,0,1,0,0,1,...,0,0,0,0,0,1,0,1,0,0
1,5575-GNVDE,0,34.0,56.95,1889.50,1,0,0,1,0,...,0,0,0,1,0,0,0,0,1,0
2,3668-QPYBK,0,2.0,53.85,108.15,1,0,0,1,0,...,0,0,0,0,0,1,0,0,1,1
3,7795-CFOCW,0,45.0,42.30,1840.75,1,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
4,9237-HQITU,0,2.0,70.70,151.65,0,0,0,1,0,...,0,0,0,0,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,0,24.0,84.80,1990.50,1,1,1,1,0,...,1,0,1,1,0,1,0,0,1,0
7039,2234-XADUH,0,72.0,103.20,7362.90,0,1,1,1,0,...,1,0,1,1,0,1,1,0,0,0
7040,4801-JZAZL,0,11.0,29.60,346.45,0,1,1,0,1,...,0,0,0,0,0,1,0,1,0,0
7041,8361-LTMKD,1,4.0,74.40,306.60,1,1,0,1,0,...,0,0,0,0,0,1,0,0,1,1


In [8]:
# Cambiando el numero se cambia el ID cliente
numero_ID = data.customerid.iloc[9]
numero_ID

'6388-TABGU'

In [31]:
# for i,element in enumerate(r.customerid):
#     if element == numero_ID: 
#         variable = r[i:i+1].copy()
#         print(type(r[i:i+1]))
        

<class 'pandas.core.frame.DataFrame'>


In [13]:
def prediccion(numero_ID):
    grilla_usuarios = frozenset(data.customerid)

    if numero_ID in grilla_usuarios:
#         model.fit(X_train,y_train)
        features = data[numero_ID == data.customerid].drop(columns=['customerid','churn_yes']).values.tolist()
        #         print(type({"valores":[feat.tolist()]}))
#         print(numero_ID)
#         r = requests.get("http://10.1.8.31:5020/predict",json={"valores":features.tolist()})
        try:
            print(features)
            result = requests.get("http://127.0.0.1:5000/predict",json={"valores":features}).json()
            return result
        except:
            print('something is wrong')
#     r.json()
    else:
        return print('no es un usuario registrado')

In [22]:
# Cambiando el numero se cambia el ID cliente
numero_ID = data.customerid.iloc[100]
numero_ID

'1091-SOZGA'

In [23]:
prediccion(numero_ID)

[[0.0, 56.0, 99.8, 5515.45, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0]]


{'El Churn predicho para el cliente es:': 0.0}

In [24]:
# Si coloco un numero de usuario inexistente pasa lo siguiente
numero_ID_incorrecto='1111-INVENTADO'

In [25]:
prediccion(numero_ID_incorrecto)

no es un usuario registrado
